In [1]:
%load_ext autoreload
%autoreload 2
import os
import os.path
import numpy as np
import sklearn
import sklearn.model_selection
import sklearn.linear_model
import sklearn.ensemble
import spacy
import sys
from sklearn.feature_extraction.text import CountVectorizer
from anchor import anchor_text
import time


In [2]:
# dataset from http://www.cs.cornell.edu/people/pabo/movie-review-data/
# Link: http://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz
def load_polarity(path='sentiment-sentences'):
    data = []
    labels = []
    f_names = ['rt-polarity.neg', 'rt-polarity.pos']
    for (l, f) in enumerate(f_names):
        for line in open(os.path.join(path, f), 'rb'):
            try:
                line.decode('utf8')
            except:
                continue
            data.append(line.strip())
            labels.append(l)
    return data, labels

Note: you must have spacy installed. Run:

        pip install spacy && python -m spacy download en_core_web_sm

If you want to run BERT, you have to install transformers and torch or tf: 

        pip install torch transformers spacy && python -m spacy download en_core_web_sm
        

In [3]:
nlp = spacy.load('en_core_web_sm')

In [4]:
data, labels = load_polarity()
train, test, train_labels, test_labels = sklearn.model_selection.train_test_split(data, labels, test_size=.2, random_state=42)
train, val, train_labels, val_labels = sklearn.model_selection.train_test_split(train, train_labels, test_size=.1, random_state=42)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
val_labels = np.array(val_labels)

In [5]:
vectorizer = CountVectorizer(min_df=1)
vectorizer.fit(train)
train_vectors = vectorizer.transform(train)
test_vectors = vectorizer.transform(test)
val_vectors = vectorizer.transform(val)

In [6]:
c = sklearn.linear_model.LogisticRegression()
# c = sklearn.ensemble.RandomForestClassifier(n_estimators=500, n_jobs=10)
c.fit(train_vectors, train_labels)
preds = c.predict(val_vectors)
print('Val accuracy', sklearn.metrics.accuracy_score(val_labels, preds))
def predict_lr(texts):
    return c.predict(vectorizer.transform(texts))

Val accuracy 0.7544910179640718


c:\users\alon\miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Explaining a prediction
use_unk_distribution=True means we will perturb examples by replacing words with UNKS

In [7]:
explainer = anchor_text.AnchorText(nlp, ['negative', 'positive'], use_unk_distribution=True)

In [8]:
np.random.seed(1)
text = 'This is a good book .'
pred = explainer.class_names[predict_lr([text])[0]]
alternative =  explainer.class_names[1 - predict_lr([text])[0]]
print('Prediction: %s' % pred)
exp = explainer.explain_instance(text, predict_lr, threshold=0.95)


Prediction: positive


Let's take a look at the anchor. Note that using this perturbation distribution, having the word 'good' in the text virtually guarantees a positive prediction

In [9]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))


Anchor: good
Precision: 1.00

Examples where anchor applies and model predicts positive:

UNK is UNK good book UNK
This is a good book .
UNK is UNK good book .
UNK UNK a good book .
UNK is UNK good UNK UNK
UNK is UNK good UNK .
This is UNK good book .
UNK UNK UNK good book UNK
UNK is UNK good UNK .
This is a good UNK UNK

Examples where anchor applies and model predicts negative:




## Using BERT

The distribution above is a bit naive, and you get a lot of sentences that don't look realistic.  
Let's use BERT to perturb the data:

In [10]:
explainer = anchor_text.AnchorText(nlp, ['negative', 'positive'], use_unk_distribution=False)

In [11]:
np.random.seed(1)
text = 'This is a good book .'
pred = explainer.class_names[predict_lr([text])[0]]
alternative =  explainer.class_names[1 - predict_lr([text])[0]]
print('Prediction: %s' % pred)
b = time.time()
exp = explainer.explain_instance(text, predict_lr, threshold=0.95, verbose=False)
print('Time: %s' % (time.time() - b))

Prediction: positive
Time: 283.98552346229553


In [12]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(only_different_prediction=True)]))

Anchor: good AND book
Precision: 0.95

Examples where anchor applies and model predicts positive:

book ##marks a good book series
it is a good book .
bold = certified good book seller
It is a good book ?
dear ##ing them good book .
it is a good book .
future is a good book ?
https : One good book reviews
he is a good book .
In here a good book ;

Examples where anchor applies and model predicts negative:

there is no good book ##ners
can any single good book ?
t ##was a good book before
there is no good book there
now here goes good book .
everything is a good book .
this is a good book edition
poor chances for good book .
everything is a good book !
here comes a good book ##let


Let's take a look at the partial anchor 'good' to see why it's not sufficient in this case


In [13]:
print('Partial anchor: %s' % (' AND '.join(exp.names(0))))
print('Precision: %.2f' % exp.precision(0))
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))

Partial anchor: good
Precision: 0.83

Examples where anchor applies and model predicts positive:

sleep is not good enough ;
that lasted a good hour .
there is always good luck :
wheat is a good crop .
1970 / 71 good ##wood ##cut
all for a good mortal .
thou wit a good favour !
this is very good information because
photography is generally good quality .
You make a good fortune .

Examples where anchor applies and model predicts negative:

Their names meant good people .
anything else provided good fortune .
This is very good video ##graphy
love is everything good and bad
multiplayer is a good idea .
God is doing good deeds …
: wins by good ##mans loss
he gave a good appearance .
your conception a good idea .
This is now good news !


Note how the examples are much more realistic than just using UNKS.

We are sampling from BERT sequentially above (one mask at a time), to get sentences that are more coherent.  
If you want to do a single BERT pass per sample, you can set `onepass=True`. You may lose some coherence, but it will be much faster.

In [14]:
np.random.seed(1)
text = 'This is a good book .'
pred = explainer.class_names[predict_lr([text])[0]]
alternative =  explainer.class_names[1 - predict_lr([text])[0]]
print('Prediction: %s' % pred)
b = time.time()
exp = explainer.explain_instance(text, predict_lr, threshold=0.95, verbose=False, onepass=True)
print('Time: %s' % (time.time() - b))

Prediction: positive
Time: 9.157501697540283


In [15]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(only_different_prediction=True)]))

Anchor: good AND book
Precision: 0.96

Examples where anchor applies and model predicts positive:

her writing a good book .
what teen not good book award
• ##r reads good book ?
< write a good book ?
award / say good book ##keeper
jack is a good book !
1971 chapter for good book signing
2000 : a good book ##worm
• ##with A good book award
It was a good book .

Examples where anchor applies and model predicts negative:

me thy say good book .
everything was a good book .
hardly for a good book .
this merit her good book .
everything is a good book .
everything is a good book cover
he me a good book .
no all made good book .
chapter excellent awful good book .
Everything – a good book !


Let's take a look at the partial anchor 'good' to see why it's not sufficient in this case


In [16]:
print('Partial anchor: %s' % (' AND '.join(exp.names(0))))
print('Precision: %.2f' % exp.precision(0))
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=2, only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=2, only_different_prediction=True)]))

Partial anchor: good
Precision: 0.85

Examples where anchor applies and model predicts positive:



IndexError: list index out of range

## See a visualization of the anchor with examples and etc (won't work if you're seeing this on github)

In [ ]:
exp.show_in_notebook()